In [0]:
import pandas as pd

In [2]:
! pip install category_encoders

In [3]:
train_df = pd.read_csv("train_data.csv")
train_df.shape

(1799805, 6)

In [4]:
train_df.head()

,date_hour,area_name,date_accident,hour_time,no_of_accidents,is_accident
0,2010-01-01 00:00:00,Olympic,2010-01-01 00:00:00,0,0.0,0
1,2010-01-01 01:00:00,Olympic,2010-01-01 00:00:00,100,1.0,1
2,2010-01-01 02:00:00,Olympic,2010-01-01 00:00:00,200,0.0,0
3,2010-01-01 03:00:00,Olympic,2010-01-01 00:00:00,300,1.0,1
4,2010-01-01 04:00:00,Olympic,2010-01-01 00:00:00,400,0.0,0


In [5]:
#Baseline Prediction for majority classsifer
train_df['is_accident'].value_counts(normalize=True)

0    0.771062
1    0.228938
Name: is_accident, dtype: float64

# Will an accident happen in a area of Los Angeles in the next one hour


### Base Line Accuracy

##### With 77% accuracy we can predict that an accident will not happen in a area within the next one hour

##### As baseline accuracy is high, we will be using AUC ROC metric for Model performance of this classification problem

In [0]:
#Converting columns to datetime
train_df['date_hour']=pd.to_datetime(train_df['date_hour'])

In [0]:
area_names = train_df.area_name.unique()

In [8]:
area_names

array(['Olympic', 'Southeast', 'Foothill', 'Hollywood', 'Mission',
       'Northeast', 'Newton', 'Central', 'N Hollywood', 'Southwest',
       'Harbor', 'Devonshire', 'Pacific', 'Rampart', 'Wilshire',
       '77th Street', 'West LA', 'Topanga', 'Van Nuys', 'West Valley',
       'Hollenbeck'], dtype=object)

In [9]:
#Frequency of accidents in Los Angeles in any given hour
train_df['no_of_accidents'].value_counts(normalize=True)

0.0     7.710619e-01
1.0     1.896428e-01
2.0     3.343084e-02
3.0     5.037212e-03
4.0     7.050764e-04
5.0     9.667714e-05
6.0     1.777970e-05
7.0     4.444926e-06
8.0     1.666847e-06
14.0    5.556157e-07
12.0    5.556157e-07
11.0    5.556157e-07
Name: no_of_accidents, dtype: float64

In [0]:
#Creating features based on no of accidents happened in the past hours in that area
empty_df = []
laghours = [4,12,24,168,720,2880,8760]
for area in area_names:
    train_area = train_df[train_df.area_name==area].copy()
    train_area['no_past1hour']=train_df['no_of_accidents'].shift(1)
    for lhour in laghours:
        name = 'past_'+str(lhour)+'hours'
        train_area[name]=train_area['no_past1hour'].rolling(lhour).sum()
    empty_df.append(train_area)

In [11]:
#Final data after creating features
train = pd.concat(empty_df)
train.shape

(1799805, 14)

In [0]:
#Filling all Nan values with -1
train = train.fillna(-1)

In [0]:
#Adding more features to the data
train['day']=train['date_hour'].dt.day
train['month']=train['date_hour'].dt.month
train['dayofweek']=train['date_hour'].dt.dayofweek
train['year']=train['date_hour'].dt.year

In [14]:
#Removing No of accidents as that is a leakage feature and date_hour as it is equal to a index
train = train.drop(columns=['no_of_accidents'])
train.shape

(1799805, 17)

In [15]:
train.tail()

,date_hour,area_name,date_accident,hour_time,is_accident,no_past1hour,past_4hours,past_12hours,past_24hours,past_168hours,past_720hours,past_2880hours,past_8760hours,day,month,dayofweek,year
1799800,2019-10-11 20:00:00,Hollenbeck,2019-10-11 00:00:00,2000,0,0.0,0.0,0.0,2.0,28.0,155.0,641.0,2020.0,11,10,4,2019
1799801,2019-10-11 21:00:00,Hollenbeck,2019-10-11 00:00:00,2100,0,0.0,0.0,0.0,2.0,28.0,155.0,640.0,2020.0,11,10,4,2019
1799802,2019-10-11 22:00:00,Hollenbeck,2019-10-11 00:00:00,2200,0,0.0,0.0,0.0,2.0,28.0,155.0,640.0,2020.0,11,10,4,2019
1799803,2019-10-11 23:00:00,Hollenbeck,2019-10-11 00:00:00,2300,0,0.0,0.0,0.0,2.0,28.0,154.0,640.0,2020.0,11,10,4,2019
1799804,2019-10-12 00:00:00,Hollenbeck,2019-10-12 00:00:00,0,0,0.0,0.0,0.0,2.0,28.0,154.0,640.0,2020.0,12,10,5,2019


In [0]:
#Creating a train,valid and test dataset based on time slicing
#Train set will be data before 2018
#Valid set will be data for year 2018
#Test set will be data for year 2019

In [0]:
target = 'is_accident'

In [18]:
train_set = train[train.date_hour<'2018-01-01']
valid_set = train[(train.date_hour>='2018-01-01')&(train.date_hour<'2019-01-01')]
test_set = train[train.date_hour>='2019-01-01']
print("Train :",train_set.shape[0])
print("Valid :",valid_set.shape[0])
print("Valid :",test_set.shape[0])

Train : 1472688
Valid : 183960
Valid : 143157


In [0]:
#separating target and training data
y_train = train_set[target]
train_set = train_set.drop(columns=target)
y_valid = valid_set[target]
valid_set = valid_set.drop(columns=target)
y_test = test_set[target]
test_set = test_set.drop(columns=target)

In [0]:
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline

In [0]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestClassifier(n_estimators=100,min_samples_leaf=2)
)

In [0]:
features = train_set.columns.drop('date_hour')

In [23]:
pipeline.fit(train_set[features],y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['area_name', 'date_accident'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'area_name',
                                          'data_type': dtype('O'),
                                          'mapping': Olympic         1
Southeast       2
Foothill        3
Hollywood       4
Mission         5
Northeast       6
Newton          7
Central         8
N Hollywood     9
Southwest      10
Harbor         11
Devonshire     12
Pacific        13
Rampa...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_

In [0]:
from sklearn.metrics import roc_auc_score

In [25]:
#roc auc score for valid set
roc_auc_score(y_valid,pipeline.predict(valid_set[features]))

0.5079310956125171

In [26]:
#roc auc score for test set
roc_auc_score(y_test,pipeline.predict(test_set[features]))

0.5078336215651406